# Stock Price Predictor using Naive Bayes
This notebook predicts stock movement direction using a Naive Bayes classifier.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import streamlit as st
import matplotlib.pyplot as plt

## Input Stock Symbol and Fetch Data

In [2]:

# Input stock symbol and date range
stock = st.text_input('Enter Stock Symbol', 'GOOG')
start = '2010-01-01'
end = '2023-12-31'

# Download stock data
data = yf.download(stock, start, end)

st.subheader('Stock Data')
st.write(data)

# Handle cases where 'Adj Close' might not be present
if 'Adj Close' in data.columns:
    price_column = 'Adj Close'
else:
    price_column = 'Close'

2025-01-22 08:00:30.588 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 08:00:30.588 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 08:00:30.588 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 08:00:30.588 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 08:00:30.592 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-01-22 08:00:30.592 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

## Feature Engineering

In [3]:
# Feature engineering
data['Return'] = data[price_column].pct_change()
data['MA50'] = data[price_column].rolling(window=50).mean()
data['MA200'] = data[price_column].rolling(window=200).mean()
data['Volatility'] = data[price_column].rolling(window=20).std()
data['Direction'] = np.where(data['Return'] > 0, 1, 0)  # Direction as target
data = data.dropna()

# Prepare features and target
X = data[['MA50', 'MA200', 'Volatility']]
y = data['Direction']

## Train-Test Split and Model Training

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Naive Bayes model
model = GaussianNB()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

## Visualization: Moving Averages

In [5]:
# Visualization: Moving Averages
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
plt.figure(figsize=(10, 6))
plt.plot(data[price_column], label='Actual Price', color='blue')
plt.plot(data['MA50'], label='50-Day MA', color='red')
plt.plot(data['MA200'], label='200-Day MA', color='green')
plt.title('Price and Moving Averages')
plt.legend()
plt.show()

## Prediction Accuracy

In [6]:
# Display prediction metrics
#accuracy = np.mean(y_pred == y_test) * 100
#st.subheader('Prediction Accuracy')
#st.write(f'Accuracy: {accuracy:.2f}%')

## Predicted vs Actual Direction

In [7]:
# Plot Predictions
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual Direction', color='red')
plt.plot(y_pred, label='Predicted Direction', color='green', alpha=0.7)
plt.title(f'{stock} Direction Prediction')
plt.xlabel('Samples')
plt.ylabel('Direction (1 = Up, 0 = Down)')
plt.legend()
plt.show()